In [ ]:
import sentence_transformers

In [1]:
import numpy as np

In [ ]:
#from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
#model = SentenceTransformer("all-MiniLM-L6-v2")

# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "The weather is not lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

# 2. Calculate embeddings by calling model.encode()
embeddings = model.encode(sentences)
print(embeddings.shape)
# [3, 384]

# 3. Calculate the embedding similarities
similarities = model.similarity(embeddings, embeddings)
print(similarities)
# tensor([[1.0000, 0.6660, 0.1046],
#         [0.6660, 1.0000, 0.1411],
#         [0.1046, 0.1411, 1.0000]])

(4, 384)
tensor([[1.0000, 0.8786, 0.6660, 0.1046],
        [0.8786, 1.0000, 0.5731, 0.1073],
        [0.6660, 0.5731, 1.0000, 0.1411],
        [0.1046, 0.1073, 0.1411, 1.0000]])


Eucledian: $\|x-y\|^2=\sum\limits_{i=1}^{d}(x_i-y_i)^2 = \|x\|^2 + \|y\|^2 -2(x,y)$

Cos:  $\frac{(x,y)}{\|x\| \|y\|}$

Dot product: $(x,y)=x_1y_1+\ldots +x_dy_d$

$\tilde{x}=\begin{bmatrix} -2x\\ \|x\|^2 \\1\end{bmatrix}, \tilde{y}=\begin{bmatrix} y\\ 1\\ \|y\|^2\end{bmatrix}$

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 37.9 MB/s eta 0:00:00


In [4]:
import faiss

In [5]:
data = np.random.rand(100_000, 384).astype("float32")
query = np.random.rand(100, 384).astype("float32")

In [6]:
index = faiss.IndexFlatL2(384)

In [7]:
index.add(data)

In [8]:
from tqdm.auto import trange

In [9]:
for i in trange(len(query)):
    index.search(query[i, None], 10)

  0%|          | 0/100 [00:00<?, ?it/s]

In [14]:
reference = np.zeros(len(query), dtype="int64")
for i in trange(len(query)):
    _dist, best_ids = index.search(query[i, None], 10)
    reference[i] = best_ids[0][0]

  0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
num_neighbors = 64
index2 = faiss.IndexHNSW (384, num_neighbors)

In [26]:
index2.add(data)

In [13]:
for i in trange(len(query)):
    index2.search(query[i, None], 10)

  0%|          | 0/100 [00:00<?, ?it/s]

In [32]:
index2.hnsw.efSearch = 80

In [35]:
index2.hnsw.efConstruction=60

In [36]:
approx = np.zeros(len(query), dtype="int64")
for i in trange(len(query)):
    _dist, best_ids = index2.search(query[i, None], 10)
    approx[i] = best_ids[0][0]

  0%|          | 0/100 [00:00<?, ?it/s]

In [37]:
(reference == approx).mean()

np.float64(0.5)